In [1]:
import sys
import os

In [2]:
sys.path.append(os.getcwd())

In [3]:
from model_platform.src.operationalization.anomaly.profile_anomaly.batch_score import *

In [4]:
data_source = "wgtraffic"

log_path = get_log_path(data_source, "profile_anomaly_batch_score")
logger = configure_logger(logger_name="profile_anomaly_batch_score", log_path=log_path, log_level=LOG_LEVEL)

logger.info(data_source)

app_name = data_source + "_profile_anomaly_batch_score"
spark = SparkSession.builder.appName(app_name).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel(PYSPARK_LOGLEVEL)

2019-04-08 20:26:49.954 INFO <ipython-input-4-1d2f8359921f> - <module>: wgtraffic


In [28]:
profile_model_dict = load_profile_anomaly_model(spark=spark, data_source=data_source)
profile_model_dict

{'ip': {'pyspark_kmeans': {'hour': <model_platform.src.model.anomaly.profile_model.cluster_profile_model.ClusterProfileModel at 0x122b2bc50>,
   'day': <model_platform.src.model.anomaly.profile_model.cluster_profile_model.ClusterProfileModel at 0x122b2eef0>,
   'week': <model_platform.src.model.anomaly.profile_model.cluster_profile_model.ClusterProfileModel at 0x122a13f28>,
   'month': <model_platform.src.model.anomaly.profile_model.cluster_profile_model.ClusterProfileModel at 0x122b34128>},
  'sklearn_isolationforest': {'hour': <model_platform.src.model.anomaly.profile_model.isolation_forest_profile_model.IsolationForestProfileModel at 0x122b38978>,
   'day': <model_platform.src.model.anomaly.profile_model.isolation_forest_profile_model.IsolationForestProfileModel at 0x122b44f28>,
   'week': <model_platform.src.model.anomaly.profile_model.isolation_forest_profile_model.IsolationForestProfileModel at 0x122b44dd8>,
   'month': <model_platform.src.model.anomaly.profile_model.isolation_fo

In [29]:
from model_platform.src.operationalization.anomaly.event_anomaly.batch_score import load_event_anomaly_model
event_model_dict = load_event_anomaly_model(spark=spark, data_source="wgtraffic")
event_model_dict

{'sklearn_isolationforest': <model_platform.src.model.anomaly.event_model.isolation_forest_event_model.IsolationForestEventModel at 0x10394dc18>,
 'sklearn_oneclasssvm': <model_platform.src.model.anomaly.event_model.oneclasssvm_event_model.OneClassSVMEventModel at 0x122dc4518>}

In [36]:
mod = profile_model_dict["ip"]["sklearn_isolationforest"]

In [37]:
sc.broadcast(mod)

In [38]:


data_path = USER_PROFILE_DATA_PATH.format(data_source=data_source, entity_type=IP_ENTITY_TYPE,
                                          anomaly_type=PROFILE_ANOMALY_TYPE,
                                          time_window="hour")
data_store = HDFSDataStore(spark=spark)
input_sdf = data_store.read_spark_df_from_data_store(data_path=data_path, data_source=data_source,
                                                     entity_type=IP_ENTITY_TYPE,
                                                     anomaly_type=PROFILE_ANOMALY_TYPE,
                                                     time_window="hour")




In [ ]:
input_sdf.show()
logger.info(input_sdf.count())
df = get_profile_anomaly_score(spark=spark, data_source=data_source, model_dict=model_dict, input_sdf=input_sdf, time_window="hour",
                                  entity_type=IP_ENTITY_TYPE)
logger.info(format(df.head()))

In [39]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

In [40]:
import random

In [41]:
def pre_process_info_1():
    return random.choice(list(range(10)))

udf_pre_process_info_1 = udf(pre_process_info_1, IntegerType())


In [42]:
input_sdf = input_sdf.withColumn("info_1", udf_pre_process_info_1())


In [43]:
input_sdf.show()

+-------------+------+------+------+------+------+
|       src_ip|wgcat1|wgcat2|wgnum1|wgnum2|info_1|
+-------------+------+------+------+------+------+
|    54.85.1.3| False|  True|   242|   473|     5|
|   54.85.5.15| False|  True|   297|   503|     1|
|  54.85.10.16|  True| False|   235|   496|     2|
|  54.85.20.40|  True| False|   235|   466|     9|
|  54.85.23.10| False| False|   257|   422|     3|
|  54.85.23.16| False|  True|   198|   463|     5|
|    54.85.9.2| False| False|   219|   489|     7|
|    54.85.9.5| False|  True|   222|   437|     4|
|  54.85.12.37| False| False|   293|   473|     5|
|  54.85.16.11| False| False|   251|   437|     6|
|   54.85.17.2| False|  True|   233|   483|     7|
|   54.85.17.9|  True| False|   215|   491|     3|
|144.160.7.102|  True|  True|   231|   471|     2|
|  54.85.21.33| False| False|   231|   535|     4|
|  54.85.22.18|  True| False|   249|   448|     9|
|   54.85.2.28|  True| False|   274|   443|     2|
|   54.85.2.32|  True|  True|  

In [44]:
input_sdf = input_sdf.repartition(10,"info_1")
input_sdf.rdd.getNumPartitions()

10

In [45]:
entity_type = "ip"
time_window = "day"

In [46]:
dic = {"hi":1}

In [82]:
import pandas as pd
def process(x):
    rows = list(x)
#     df1 = pd.DataFrame(data=[[x["wgcat1"], x["wgcat2"], x["wgnum1"], x["wgnum2"]]],
#                               columns=["wgcat1", "wgcat2", "wgnum1", "wgnum2"])
    df1 = pd.DataFrame(rows,columns=["wgcat1", "wgcat2", "wgnum1", "wgnum2"])
#     cc = mod["hour"].score(df1)
#     df1["PAS_isolation"] = [1 for i in range(len(df1))]
#     lll = df1["PAS_isolation"].values.tolist()
    return [df1]



In [83]:
import pandas as pd

rdd = sc.parallelize([
    {"x": 1, "y": -1}, 
    {"x": -3, "y": 0},
    {"x": -0, "y": 4}
])

def combine(iter):
    rows = list(iter)
    return [pd.DataFrame(rows)] if rows else []

rdd.mapPartitions(combine).first()

,x,y
0,1,-1


In [84]:
x = input_sdf.rdd.mapPartitions(process)
x.take(1)

[Empty DataFrame
 Columns: [wgcat1, wgcat2, wgnum1, wgnum2]
 Index: []]

In [68]:
def filterOut2FromPartion(list_of_lists):
    final_iterator = []
    for sub_list in list_of_lists:
        final_iterator.append( [x for x in sub_list if x != 2])
    return iter(final_iterator)
data = sc.parallelize([ [1, 2, 3], [3, 2, 4], [5, 2, 7] ] )
filtered_lists = data.mapPartitions(filterOut2FromPartion)
filtered_lists.take(2)

[[1, 3], [3, 4]]

In [64]:
x.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 331.0 failed 1 times, most recent failure: Lost task 0.0 in stage 331.0 (TID 394, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Applications/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/Applications/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Applications/spark-2.1.1-bin-hadoop2.7/python/pyspark/rdd.py", line 2408, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Applications/spark-2.1.1-bin-hadoop2.7/python/pyspark/rdd.py", line 345, in func
    return f(iterator)
  File "<ipython-input-62-07303ad4b18e>", line 2, in process
Type

In [58]:
import pandas as pd

In [61]:
m= y[0]

In [63]:
m.asDict().keys()

dict_keys(['src_ip', 'wgcat1', 'wgcat2', 'wgnum1', 'wgnum2', 'info_1'])

In [68]:
kk = pd.DataFrame(y,columns=m.asDict().keys())

In [73]:
cc = model_dict[entity_type][SKLEARN_ONECLASS_SVM_MODEL][time_window].score(kk)
kk["PAS_isolation"] = cc["PAS"]
lll = kk["PAS_isolation"].values.tolist()

In [ ]:
data_tuples = list(zip(Month,Days))
data_tuples
[('Jan', 31), ('Apr', 30), ('Mar', 31), ('June', 30)]